In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
def import_sys():
    import sys
    sys.path.append('..')
    sys.path.append('../..')
import_sys()

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # pylint: disable=invalid-name

In [2]:
import glob, os

import errudite
from errudite.io import DatasetReader
from errudite.predictors import Predictor
from errudite.targets.instance import Instance
from errudite.targets.label import Label
from errudite.utils import accuracy_score, normalize_file_path

DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/out_of_domain_devs")
sample_size=100
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz")))
reader = DatasetReader.by_name("mrqa")(
    cache_folder_path=f"~/datasets/caches/dataset_debug/mrqa-{sample_size}")
instances = reader.read(file_path, sample_size=sample_size)

MODEL_FOLDER = normalize_file_path("~/datasets/models/mrqa/")
predictor = Predictor.by_name("mrqa")(
    name="mrqa_path", 
    model_path=os.path.join(MODEL_FOLDER, "mrqa_bert_base.gz"))

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
INFO:errudite.utils.file_utils:Errudite cache folder selected: /Users/tongshuangwu/datasets/caches/dataset_debug/mrqa-100
INFO:errudite.io.dataset_reader:Reading instances from lines in file at: /Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/BioASQ.jsonl.gz,/Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/RACE.jsonl.gz,/Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/DROP.jsonl.gz,/Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/TextbookQA.jsonl.gz,/Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/

In [3]:
import pandas as pd
from tqdm import tqdm
predictors = { p.name: p for p in [predictor] }
predictions = { p: [] for p in predictors }
logger.info("Running predictions....")
for instance in tqdm(instances):
    instance_predictions = []
    for predictor in predictors.values():
        prediction = Predictor.by_name("qa_task_class").model_predict(
            predictor, 
            instance.question, 
            instance.context, 
            instance.groundtruths)
        instance_predictions.append(prediction)
        predictions[predictor.name].append(prediction)
    instance.set_entries(predictions=instance_predictions)
for predictor in predictors.values():
    predictor.evaluate_performance(instances)
print(pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ]))
Instance.build_instance_hashes(instances)

INFO:__main__:Running predictions....
100%|██████████| 102/102 [02:37<00:00,  2.27s/it]

         f1  predictor
0  0.582143  mrqa_path


({InstanceKey(qid='2cf03749226444fea30c672c678b3d63', vid=0): Instance [InstanceKey(qid='2cf03749226444fea30c672c678b3d63', vid=0)],
  InstanceKey(qid='b0f3562cb2574620ba46ae33c88d65b3', vid=0): Instance [InstanceKey(qid='b0f3562cb2574620ba46ae33c88d65b3', vid=0)],
  InstanceKey(qid='b7cd2f0a380d4d14b488e20ddf88b204', vid=0): Instance [InstanceKey(qid='b7cd2f0a380d4d14b488e20ddf88b204', vid=0)],
  InstanceKey(qid='733e977ad4f14caf9148b8dbb151fd8c', vid=0): Instance [InstanceKey(qid='733e977ad4f14caf9148b8dbb151fd8c', vid=0)],
  InstanceKey(qid='c3addb03d57c47208e8a84201cdf7608', vid=0): Instance [InstanceKey(qid='c3addb03d57c47208e8a84201cdf7608', vid=0)],
  InstanceKey(qid='26a83e722adc4516a5e0f6a8f7c8caaa', vid=0): Instance [InstanceKey(qid='26a83e722adc4516a5e0f6a8f7c8caaa', vid=0)],
  InstanceKey(qid='b7b765a669484522bfdfa013ced2bc9d', vid=0): Instance [InstanceKey(qid='b7b765a669484522bfdfa013ced2bc9d', vid=0)],
  InstanceKey(qid='1907600356cb4afeb53267a9f83291d7', vid=0): Instanc

In [3]:
import pandas as pd
pd.DataFrame([ {"predictor": p.name, "f1": p.perform["f1"] } for p in predictors.values() ])

NameError: name 'predictors' is not defined

In [5]:
TRAIN_DATASET_FOLDER = normalize_file_path("~/datasets/raw_data/mrqa/trains")
sample_size=100
file_path = ",".join(glob.glob(os.path.join(DATASET_FOLDER, "*.jsonl.gz"))[:2])
reader.count_vocab_freq(file_path)

INFO:errudite.io.dataset_reader:Computing vocab frequency from file at: /Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/BioASQ.jsonl.gz,/Users/tongshuangwu/datasets/raw_data/mrqa/out_of_domain_devs/RACE.jsonl.gz
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

INFO:root:
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-181d9ff5b766>", line 4, in <module>
    reader.count_vocab_freq(file_path)
  File "../errudite/io/dataset_reader.py", line 124, in count_vocab_freq
    target_dicts = self._read(file_path, lazy=True, sample_size=None)
  File "../errudite/io/mrqa_reader.py", line 59, in _read
    for cidx, example in enumerate(dataset['file_handle']):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/gzip.py", line 373, in readline
    self._check_not_closed()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tongshuangwu/sourcetree/errudite_dataset_debug/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2039, 

KeyboardInterrupt: 